In [ ]:
import io, os, sys, ssl, time, json, socket, struct, datetime, threading
import requests, serial, repl, picamera, pigpio

server_ip  = 'yzlab3.chem.nyu.edu'#'guoxiaokang.net'
img_port   = 54321
ctrl_port  = 54322
flask_port = 54323
basic_auth_username = 'Hal?ou'
basic_auth_password = 'Wode*%^!()'

server_cert = f'{os.environ["HOME"]}/openssl/server.crt' 
client_cert = f'{os.environ["HOME"]}/openssl/rover.crt'
client_key  = f'{os.environ["HOME"]}/openssl/rover.key'

# Establish connection with server
context = ssl.create_default_context(ssl.Purpose.SERVER_AUTH, cafile=server_cert)
context.load_cert_chain(certfile=client_cert, keyfile=client_key)

In [ ]:
class Sensor(): 
    def __init__(self):
        self.data = {}
        self.Temperature = 0 
        self.interval = 1
        self.serial_port = '/dev/ttyS0'
        self.usb_port = '/dev/ttyACM0'
        #threading.Thread(target=self.gps).start()   
        threading.Thread(target=self.microbit).start()
        time.sleep(3)
    
    def gps(self): 
        s = serial.Serial(port=self.serial_port, baudrate=9600, timeout=3)
        while True:
            gprmc = s.readline() # You can get GPS data if the on-board LED blinks.
            if gprmc[0:6] == b'$GPRMC': self.data['P'] = gprmc     
            time.sleep(self.interval)
            
    def microbit(self):             
        s = serial.Serial(port     = self.usb_port, 
                          baudrate = 115200, 
                          parity   = serial.PARITY_NONE, 
                          stopbits = serial.STOPBITS_ONE, 
                          timeout  = 0)
        self.repl = repl.REPL(s)
        self.repl.to_raw()
        while True:
            self.repl.send_command("print(temperature())") 
            self.Temperature = self.repl.wait_response().rstrip()
            self.repl.send_command("print(accelerometer.get_values())") 
            self.data['a'] = self.repl.wait_response()
            self.repl.send_command("print(compass.heading())") 
            self.data['N'] = self.repl.wait_response()
            time.sleep(self.interval)

sensor = Sensor() 

In [ ]:
camera = picamera.PiCamera(resolution=(640, 480)) 
camera.rotation = 270
camera.start_preview() 
time.sleep(2)

# https://picamera.readthedocs.io/en/release-1.13/recipes1.html#capturing-to-a-network-stream
# https://picamera.readthedocs.io/en/release-1.13/recipes2.html#web-streaming

def imgd():
    while True:
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            img_ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
            img_ss.connect((server_ip, img_port)) 
            img_io = img_ss.makefile('wb')
            stream = io.BytesIO()
            for foo in camera.capture_continuous(stream, 'jpeg'): 
                dt = datetime.datetime.now().strftime('@ %H:%M:%S %b %d %Y')
                camera.annotate_text = f'Celsius:{sensor.Temperature.rstrip()} {dt}'  
                img_io.write(struct.pack('<L', stream.tell()))
                img_io.flush()
                stream.seek(0)
                img_io.write(stream.read()) 
                img_io.flush()
                stream.seek(0)
                stream.truncate()
                time.sleep(1)
        except Exception as e:
            print(e)
            time.sleep(15)
    
threading.Thread(target=imgd).start() 

In [ ]:
!sudo pkill pigpiod
!sudo pigpiod -p 8889 ### Start PiGPIO Server / Deamon          http://abyz.me.uk/rpi/pigpio/pigpiod.html
pi = pigpio.pi(port=8889) ### Start PiGPIO Client 
servo = lambda l1, l2: [pi.set_servo_pulsewidth(_1, _2) for _1, _2 in zip(l1, l2)]
motor = lambda l1, l2: [pi.set_PWM_dutycycle(   _1, _2) for _1, _2 in zip(l1, l2)]
drive_pin, reverse_pin      = 16, 20 
elevation_pin, rotation_pin = 26, 19
top_pin, bottom_pin         =  6, 13  # {top:900 ~ 1800 , bottom: 1500 ~ 2350}

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ctrl_ss = context.wrap_socket(s, server_side=False, server_hostname=server_ip)
ctrl_ss.connect((server_ip, ctrl_port))  

def ctrd():      
    l = struct.calcsize('<Hi')
    while True:
        code, value = struct.unpack('<Hi', ctrl_ss.recv(l))
        if   code ==  0: servo([top_pin, bottom_pin], [value, 3300-value]) 
        elif code ==  2: pi.set_servo_pulsewidth(rotation_pin , value) 
        elif code ==  5: pi.set_servo_pulsewidth(elevation_pin, value) 
        elif code == 10: motor([drive_pin, reverse_pin], [0, value])  # break
        elif code ==  9: motor([reverse_pin, drive_pin], [0, value])  # gas
        elif code ==304 and value: servo([top_pin, bottom_pin], [1350, 1950])  
            
threading.Thread(target=ctrd).start()

In [ ]:
def antiShakingd():      
    while True: 
        servo([top_pin, bottom_pin, rotation_pin, elevation_pin], [0, 0, 0, 0]) 
        time.sleep(1)
threading.Thread(target=antiShakingd).start()

In [ ]:
def reporter():
    while True:
        time.sleep(1)
        requests.post(f'https://{server_ip}:{flask_port}/sensor', 
                      data=json.dumps(sensor.data), 
                      auth=(basic_auth_username, basic_auth_password)) 
#reporter()
threading.Thread(target=reporter).start()